In [0]:
%sql
USE CATALOG mvp;
USE SCHEMA gold;

### Construção das entidades dimensões:

In [0]:
df = spark.sql("select * from silver.voos_tipado")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

def moda_por_coluna(df, chave, coluna):
    w = Window.partitionBy(chave).orderBy(F.desc("count"))
    
    return (
        df.groupBy(chave, coluna)
          .count()
          .withColumn("rn", F.row_number().over(w))
          .filter("rn = 1")
          .select(chave, coluna)
    )

In [0]:
# dim_aeronave
print("dim_aeronave:")
column_id = "id_equipamento"
colunas = ["ds_modelo", "ds_matricula", "sg_equipamento_icao"]
dim_aeronave = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_aeronave = moda_df if dim_aeronave is None else dim_aeronave.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_aeronave.groupBy(column_id).count().where("count > 1").count())
dim_aeronave.limit(5).display()


dim_aeronave:
IDs repetidos: 0


id_equipamento,ds_modelo,ds_matricula,sg_equipamento_icao
14,AIRBUS A319,TME,A319
15,AIRBUS A320-100/200,TYA,A320
16,AIRBUS A321-100/200,MXD,A321
19,AIRBUS A330-200,AIW,A332
26,AIRBUS A350-900,AOY,A359


In [0]:

# dim_aeroporto
print("dim_aeroporto:")
aeroportos = df.select([
    F.col("id_aerodromo_destino").alias("id_aerodromo"),
    F.col("nm_aerodromo_destino").alias("nm_aerodromo"),
    F.col("nm_continente_destino").alias("nm_continente"),
    F.col("nm_municipio_destino").alias("nm_municipio"),
    F.col("nm_pais_destino").alias("nm_pais"),
    F.col("nm_regiao_destino").alias("nm_regiao"),
    F.col("sg_iata_destino").alias("sg_iata"),
    F.col("sg_icao_destino").alias("sg_icao"),
    F.col("sg_uf_destino").alias("sg_uf")
]).union(df.select([
    F.col("id_aerodromo_origem").alias("id_aerodromo"),
    F.col("nm_aerodromo_origem").alias("nm_aerodromo"),
    F.col("nm_continente_origem").alias("nm_continente"),
    F.col("nm_municipio_origem").alias("nm_municipio"),
    F.col("nm_pais_origem").alias("nm_pais"),
    F.col("nm_regiao_origem").alias("nm_regiao"),
    F.col("sg_iata_origem").alias("sg_iata"),
    F.col("sg_icao_origem").alias("sg_icao"),
    F.col("sg_uf_origem").alias("sg_uf")
]))
column_id = "id_aerodromo"
colunas = [
    "nm_aerodromo",
    "nm_continente",
    "nm_municipio",
    "nm_pais",
    "nm_regiao",
    "sg_iata",
    "sg_icao",
    "sg_uf"
]
dim_aeroporto = None

for col in colunas:
    moda_df = moda_por_coluna(aeroportos, column_id, col)
    dim_aeroporto = moda_df if dim_aeroporto is None else dim_aeroporto.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_aeroporto.groupBy(column_id).count().where("count > 1").count())
dim_aeroporto.limit(5).display()


dim_aeroporto:
IDs repetidos: 0


id_aerodromo,nm_aerodromo,nm_continente,nm_municipio,nm_pais,nm_regiao,sg_iata,sg_icao,sg_uf
4,GENERAL JUAN N. ÁLVAREZ INTERNATIONAL AIRPORT,AMÉRICA DO NORTE,ACAPULCO/GEN.JUAN ALVAREZ,MÉXICO,null,ACA,MMAA,null
5,KOTOKA INTERNATIONAL AIRPORT,ÁFRICA,ACCRA,GANA,null,ACC,DGAA,null
10,OLHOS D´ÁGUA,AMÉRICA DO SUL,ÁGUA BOA,BRASIL,CENTRO-OESTE,GGB,SWHP,MT
13,GAUDÊNCIO MACHADO RAMOS,AMÉRICA DO SUL,ALEGRETE,BRASIL,SUL,ALQ,SSLT,RS
16,MONTE DOURADO,AMÉRICA DO SUL,ALMEIRIM,BRASIL,NORTE,MEU,SBMD,PA


In [0]:
# dim_empresa
print("dim_empresa:")
column_id = "id_empresa"
colunas = [
    "sg_empresa_icao",
    "sg_empresa_iata",
    "nm_empresa",
    "nm_pais"
] 
  
dim_empresa = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_empresa = moda_df if dim_empresa is None else dim_empresa.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_empresa.groupBy(column_id).count().where("count > 1").count())
dim_empresa.limit(5).display()


dim_empresa:
IDs repetidos: 0


id_empresa,sg_empresa_icao,sg_empresa_iata,nm_empresa,nm_pais
1000220,IPM,8I,ITAPEMIRIM TRANSPORTES AEREOS LTDA,BRASIL
1000584,ABJ,E4,ATA - AEROTÁXI ABAETÉ LTDA.,BRASIL
1000625,CQB,Q1,APUÍ TÁXI AÉREO S/A,BRASIL
1000687,ACN,2F,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),BRASIL
1000711,OMI,O1,OMNI TÁXI AÉREO S.A.,BRASIL


In [0]:
# dim_linha
print("dim_linha:")
column_id = "id_tipo_linha"
colunas = [
    "cd_tipo_linha", 
    "ds_tipo_linha", 
    "ds_natureza_tipo_linha", 
    "ds_servico_tipo_linha"
] 
  
dim_linha = None

for col in colunas:
    moda_df = moda_por_coluna(df, column_id, col)
    dim_linha = moda_df if dim_linha is None else dim_linha.join(moda_df, column_id, "left")

print("IDs repetidos:", dim_linha.groupBy(column_id).count().where("count > 1").count())
dim_linha.limit(5).display()


dim_linha:
IDs repetidos: 0


id_tipo_linha,cd_tipo_linha,ds_tipo_linha,ds_natureza_tipo_linha,ds_servico_tipo_linha
1,N,DOMÉSTICA MISTA,DOMÉSTICA,PASSAGEIRO
3,I,INTERNACIONAL MISTA,INTERNACIONAL,PASSAGEIRO


In [0]:
# dim_data
datas = df.select([
    F.col("dt_partida_real").alias("dt_voo")
]).union(df.select([
    F.col("dt_chegada_real").alias("dt_voo")
]))

column_id = "dt_voo"
datas = datas.distinct()
dim_data = datas\
    .withColumn("dt_voo", F.to_date(F.col("dt_voo")))\
    .withColumn("nr_dia", F.day("dt_voo"))\
    .withColumn("nr_mes", F.month("dt_voo"))\
    .withColumn("nr_ano", F.year("dt_voo"))\
    .withColumn("nr_semestre", F.when(F.col("nr_mes") <= 6, 1).otherwise(2))\
    .withColumn("nr_trimestre", F.ceil(F.col("nr_mes") / 3))

print("IDs repetidos:", dim_data.groupBy(column_id).count().where("count > 1").count())
dim_data.limit(5).display()
    

IDs repetidos: 0


dt_voo,nr_dia,nr_mes,nr_ano,nr_semestre,nr_trimestre
2021-03-28,28,3,2021,1,1
2022-10-28,28,10,2022,2,4
2020-03-22,22,3,2020,1,1
2020-12-25,25,12,2020,2,4
2020-07-01,1,7,2020,2,3


### Construção da tabela fato:

In [0]:
column_id = "id_basica"
fact_voo = df.select([
  "id_basica",
  "nr_voo",
  "nr_singular",
  "ds_natureza_etapa",
  "lt_combustivel", 
  "nr_assentos_ofertados",
  "kg_payload",
  "km_distancia",
  "nr_passag_pagos",
  "nr_passag_gratis", 
  "kg_bagagem_livre",
  "kg_bagagem_excesso",
  "kg_carga_paga",
  "kg_carga_gratis",
  "kg_correio",
  "nr_horas_voadas",
  "kg_peso",
  "nr_velocidade_media",
  "nr_pax_gratis_km",
  "nr_carga_paga_km",
  "nr_carga_gratis_km",
  "nr_correio_km",
  "nr_bagagem_paga_km",
  "nr_bagagem_gratis_km",
  "nr_ask",
  "nr_rpk",
  "nr_atk",
  "dt_partida_real",
  "dt_chegada_real",
  "id_tipo_linha",
  "id_empresa",
  "id_aerodromo_origem",
  "id_aerodromo_destino",
  "id_equipamento",
])


fact_voo = fact_voo.withColumnRenamed("id_aerodromo_origem", "pk_id_aerodromo_partida")
fact_voo = fact_voo.withColumnRenamed("id_aerodromo_destino", "pk_id_aerodromo_chegada")
fact_voo = fact_voo.withColumnRenamed("id_equipamento", "pk_id_equipamento")
fact_voo = fact_voo.withColumnRenamed("id_empresa", "pk_id_empresa")
fact_voo = fact_voo.withColumnRenamed("id_tipo_linha", "pk_id_tipo_linha")
fact_voo = fact_voo.withColumnRenamed("dt_partida_real", "pk_id_dt_partida")
fact_voo = fact_voo.withColumnRenamed("dt_chegada_real", "pk_id_dt_chegada")

print("IDs repetidos:", fact_voo.groupBy(column_id).count().where("count > 1").count())
fact_voo.limit(5).display()


IDs repetidos: 0


id_basica,nr_voo,nr_singular,ds_natureza_etapa,lt_combustivel,nr_assentos_ofertados,kg_payload,km_distancia,nr_passag_pagos,nr_passag_gratis,kg_bagagem_livre,kg_bagagem_excesso,kg_carga_paga,kg_carga_gratis,kg_correio,nr_horas_voadas,kg_peso,nr_velocidade_media,nr_pax_gratis_km,nr_carga_paga_km,nr_carga_gratis_km,nr_correio_km,nr_bagagem_paga_km,nr_bagagem_gratis_km,nr_ask,nr_rpk,nr_atk,pk_id_dt_partida,pk_id_dt_chegada,pk_id_tipo_linha,pk_id_empresa,pk_id_aerodromo_partida,pk_id_aerodromo_chegada,pk_id_equipamento
30530321,2014,450,DOMÉSTICA,8228,186,19800,2177,159,2,1269,3,0,0,0,null,13347,null,4354,0,0,0,6531,2762610,404922,346143,43104,2022-11-16,2022-11-16,1,1000854,626,268,505
30530325,1567,450,DOMÉSTICA,6373,186,19800,1480,172,3,1766,0,6,0,0,null,14897,null,4440,8880,0,0,0,2613680,275280,254560,29304,2022-11-30,2022-11-30,1,1000854,654,698,505
30530334,1786,450,DOMÉSTICA,3676,186,19800,946,143,18,1036,8,0,0,0,null,13119,null,17028,0,0,0,7568,980056,175956,135278,18730,2022-11-12,2022-11-12,1,1000854,122,594,505
30530332,7692,450,INTERNACIONAL,4905,176,19800,1211,146,0,2742,5,0,0,0,null,13697,null,0,0,0,0,6055,3320560,213136,176806,23977,2022-11-26,2022-11-26,3,1000854,263,138,505
30530329,1630,450,DOMÉSTICA,11033,186,19800,2697,167,3,1742,0,1348,0,0,null,15840,null,8091,3635560,0,0,0,4698170,501642,450399,53400,2022-11-10,2022-11-10,1,1000854,301,438,505


In [0]:
print("Total de registros na tabela dim_aeronave", dim_aeronave.count())
print("Total de registros na tabela dim_aeroporto", dim_aeroporto.count())
print("Total de registros na tabela dim_empresa", dim_empresa.count())
print("Total de registros na tabela dim_linha", dim_linha.count())
print("Total de registros na tabela dim_data", dim_data.count())
print("Total de registros na tabela fact_voo", fact_voo.count())

Total de registros na tabela dim_aeronave 26
Total de registros na tabela dim_aeroporto 322
Total de registros na tabela dim_empresa 14
Total de registros na tabela dim_linha 2
Total de registros na tabela dim_data 1800
Total de registros na tabela fact_voo 3254041


### Persistência das tabelas:

In [0]:
dim_aeronave.write.format("delta").mode("overwrite").saveAsTable("dim_aeronave")
dim_aeroporto.write.format("delta").mode("overwrite").saveAsTable("dim_aeroporto")
dim_empresa.write.format("delta").mode("overwrite").saveAsTable("dim_empresa")
dim_linha.write.format("delta").mode("overwrite").saveAsTable("dim_linha")
dim_data.write.format("delta").mode("overwrite").saveAsTable("dim_data")
fact_voo.write.format("delta").mode("overwrite").saveAsTable("fact_voo")

### Adicionando comentários as colunas:

In [0]:
print("Comentando a tabela dim_aeronave")

tabela = "mvp.gold.dim_aeronave"
descricao_tabela = "A tabela contém dados das aeronaves do dataframe original."
esquema = "mvp"
tabela = "gold.dim_aeronave"
colunas = [
    ("id_equipamento", "Código numérico identificador da aeronave no sistema."),
    ("ds_matricula", "Marca de matrícula da aeronave."),
    ("ds_modelo", "Descrição do modelo da aeronave."),
    ("sg_equipamento_icao", "Designador ICAO do modelo da aeronave (“Type Designator”)"),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela dim_aeronave


In [0]:
print("Comentando a tabela dim_aeroporto")

tabela = "mvp.gold.dim_aeroporto"
descricao_tabela = "A tabela contém dados dos aeroportos do dataframe original."
esquema = "mvp"
tabela = "gold.dim_aeroporto"
colunas = [
    ("id_aerodromo", "Código identificador do aeródromo no sistema."),
    ("nm_aerodromo", "Nome do aeródromo."),
    ("nm_continente", "Nome do continente em que o aeródromo fica localizado."),
    ("nm_municipio", "Nome do municipio em que o aeródromo fica localizado"),
    ("nm_pais", "Nome do pais em que o aeródromo fica localizado"),
    ("nm_regiao", "Nome da região em que o aeródromo fica localizado"),
    ("sg_iata", "Sigla IATA do aeródromo."),
    ("sg_icao", "Sigla ICAO do aeródromo."),
    ("sg_uf", "Sigla da Unidade Federativa em que o aeródromo fica localizado"),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela dim_aeroporto


In [0]:
print("Comentando a tabela dim_empresa")

tabela = "mvp.gold.dim_empresa"
descricao_tabela = "A tabela contém dados das empresas responsáveis pelos voos."
esquema = "mvp"
tabela = "gold.dim_empresa"
colunas = [
    ("id_empresa", "Código numérico identificador da empresa no sistema."),
    ("nm_empresa", "Nome da empresa aérea."),
    ("nm_pais", "Nome do país da nacionalidade da empresa aérea."),
    ("sg_empresa_iata", "Sigla IATA da empresa aérea. Refere-se ao designador da empresa de transporte aéreo obtido junto à IATA (Associação Internacional de Transporte Aéreo)."),
    ("sg_empresa_icao", "Sigla ICAO da empresa aérea. Refere-se ao designador da empresa de transporte aéreo obtido junto à OACI (Organização da Aviação Civil Internacional)."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela dim_empresa


In [0]:
print("Comentando a tabela dim_linha")

tabela = "mvp.gold.dim_linha"
descricao_tabela = "A tabela contém dados dos tipos de linhas de voo."
esquema = "mvp"
tabela = "gold.dim_linha"
colunas = [
    ("id_tipo_linha", "Código numérico identificador do tipo de linha no sistema."),  
    ("cd_tipo_linha", "Caractere que identifica o tipo de linha. O Tipo de Linha identifica o tipo de operação realizada no voo, podendo ser classificado em: a) 0 (zero) - Etapa Regular; b) 2 (dois) - Etapa Extra; c) 3 (três) -Etapa de Retorno; d) 4 (quatro) -Inclusão de Etapa; e) 6 (seis) -Etapa Não Remunerada Sem Transporte de Objetos; f) 7 (sete) -Etapa de Voo de Fretamento; g) 9(nove) -Etapa de Voo Charter;h) D -Etapa de Voo Duplicada; i) E - Etapa Não Remunerada Com Transporte de Objetos."),
    ("ds_natureza_tipo_linha", "Descrição da natureza referente ao tipo de linha (Internacional/Doméstica). Considera a natureza do voo, não apenas da etapa."),
    ("ds_servico_tipo_linha", "Descrição do serviço referente ao tipo de linha (Passageiro/Cargueiro)."),
    ("ds_tipo_linha", "Descrição do tipo de linha."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela dim_linha


In [0]:
print("Comentando a tabela dim_data")

tabela = "mvp.gold.dim_data"
descricao_tabela = "A tabela contém datas dos voos."
esquema = "mvp"
tabela = "gold.dim_data"
colunas = [
    ("dt_voo", "Data de partida ou chegada de um voo."),  
    ("nr_dia", "O dia relativo a data."),
    ("nr_mes", "O mês relativo a data."),
    ("nr_ano", "O ano relativo a data."),
    ("nr_semestre", "O semestre relativo a data."),
    ("nr_trimestre", "O trimestre relativo a data."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela dim_data


In [0]:
print("Comentando a tabela fact_voo")

tabela = "mvp.gold.fact_voo"
descricao_tabela = "A tabela contém dados de voos operados em com origem ou destino nacional."
esquema = "mvp"
tabela = "gold.fact_voo"
colunas = [
    ("id_basica", "Código numérico identificador da etapa básica no sistema."),
    ("nr_voo", "Número do voo. Refere-se ao número atribuído à operação de uma etapa ou de uma série de etapas registradas sob a mesma numeração de voo."),
    ("nr_singular", "Singularidade do Voo. Refere-se ao conjunto de caracteres que auxilia na identificação do voo, composto de letras e números escolhidos a critério da própria empresa aérea."),
    ("ds_natureza_etapa", "Descrição da natureza da etapa de voo (Internacional/Doméstica)."),
    ("lt_combustivel", "Quantidade de combustível consumida, expressa em litros."),
    ("nr_assentos_ofertados", "Número de assentos disponíveis na etapa do voo."),
    ("kg_payload", "Capacidade Payload, expressa em quilogramas."),
    ("km_distancia", "Distância, expressa em quilômetros, entre os aeródromos de origem e destino da etapa, considerando a curvatura do planeta terra."),
    ("nr_passag_pagos", "Número de passageiros que ocupam assentos comercializados ao público e que geram receita, com a compra de assentos, para a empresa de transporte aéreo."),
    ("nr_passag_gratis", "Número de passageiros que ocupam assentos comercializados ao público, mas não geram receita, com a compra de assentos, para a empresa de transporte aéreo."),
    ("kg_bagagem_livre", "Bagagem franqueada. É toda bagagem que não é considerada como excesso, expressa em quilogramas."),
    ("kg_bagagem_excesso", "Excesso de bagagem. É a quantidade total de bagagem que excede o limite de peso acordado entre a empresa aérea e o passageiro (adquirido antes ou depois da compra do bilhete), verificada no momento do despacho, expressa em quilogramas."),
    ("kg_carga_paga", "Carga paga. É a quantidade total, expressa em quilogramas, de todos os bens que tenham sido transportados na aeronave, exceto correio e bagagem, e tenham gerado receita direta ou indireta para a empresa aérea."),
    ("kg_carga_gratis", "Quantidade total, expressa em quilogramas, de todos os bens que tenham sido transportados na aeronave, exceto correio e bagagem, e não tenha gerado receitas diretas ou indiretas para a empresa aérea."),
    ("kg_correio", "Quantidade, expressa em quilogramas, de objetos transportados para atender aos operadores designados oficialmente pelo país para operar serviços postais e cumprir com as obrigações associadas decorrentes dos Atos da Universal Postal Union (UPU)."),
    ("nr_horas_voadas", "Número de horas voadas."),
    ("kg_peso", "Refere-se ao peso total carregado pela aeronave, calculado pela soma de carga, correio, passageiros e bagagem, expressos em kg. O peso dos passageiros transportados é calculado multiplicando-se a quantidade total de passageiros por 75, para empresas brasileiras. No caso das empresas estrangeiras, que não enviam dados de bagagem à ANAC, o peso total de passageiros e bagagens é estimado multiplicando-se a quantidade total de passageiros por 90."),
    ("nr_velocidade_media", "Velocidade média do voo, calculada a partir da distância e da duração do voo, expressa em km/h."),
    ("nr_pax_gratis_km", "Representa, para cada etapa básica, o produto entre a quantidade de passageiros grátis e a distância da etapa."),
    ("nr_carga_paga_km", "Refere-se ao produto entre a quantidade de carga paga (kg) e a distância da etapa básica (km)."),
    ("nr_carga_gratis_km", "Refere-se ao produto entre a quantidade de carga grátis (kg) e a distância da etapa básica (km)."),
    ("nr_correio_km", "Refere-se ao produto entre a quantidade de correio (kg) e a distância da etapa básica (km)."),
    ("nr_bagagem_paga_km", "Refere-se ao produto entre a quantidade de bagagem paga (kg) e a distância da etapa básica (km)."),
    ("nr_bagagem_gratis_km", "Refere-se ao produto entre a quantidade de bagagem grátis (kg) e a distância da etapa básica (km)."),
    ("nr_ask", "Refere-se ao volume de Assentos Quilômetros Oferecidos (Available Seat Kilometer). É calculado, em cada etapa básica, pelo produto entre o número de assentos oferecidos e a distância da etapa."),
    ("nr_rpk", "Refere-se ao volume de Passageiros Quilômetros Transportados (Revenue Passenger Kilometer). É calculado, em cada etapa básica, pelo produto entre o número de passageiros pagos e a distância da etapa."),
    ("nr_atk", "Refere-se ao volume de Toneladas Quilômetros Oferecidas (Available Tonne Kilometer), expresso em (ton x km). É calculado, em cada etapa básica, pelo produto entre a Capacidade Payload (kg) e a distância da etapa, dividido por 1000."),
    ("pk_id_dt_partida", "Chave extrangeira que indica qual a data de partida do voo."),
    ("pk_id_dt_chegada", "Chave estrangeira que indica qual a data de chegada do voo."),
    ("pk_id_tipo_linha", "Chave estrangeira que indica o tipo de linha do voo."),
    ("pk_id_empresa", "Chave estrangeira que indica a empresa do voo."),
    ("pk_id_aerodromo_partida", "Chave estrangeira que indica o aeroporto de origem do voo."),
    ("pk_id_aerodromo_chegada", "Chave estrangeira que indica o aeroporto de chegada do voo."),
    ("pk_id_equipamento", "Chave estrangeira que indica a aeronave utilizada no voo."),
]

spark.sql(f"COMMENT ON TABLE {tabela} IS '{descricao_tabela}'")

for col, comment in colunas:
    spark.sql(f"COMMENT ON COLUMN {tabela}.{col} IS '{comment}'")

Comentando a tabela fact_voo
